Imports e configuração

In [ ]:
import os
import json
import nltk
import psycopg2
import psycopg2.extras
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import HTMLResponse, JSONResponse
from pydantic import BaseModel
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import SystemMessage, HumanMessage
from langchain_unstructured import UnstructuredLoader
from dotenv import load_dotenv
import uvicorn


load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

nltk.download('punkt')

app = FastAPI()


Conexão com o banco de dados e criação de tabelas

In [ ]:
def connect_db():
    try:
        return psycopg2.connect(
            host=os.getenv("PG_HOST", "localhost"),
            dbname=os.getenv("PG_DB", "pgvector_db"),
            user=os.getenv("PG_USER", "LFP"),
            password=os.getenv("PG_PASS", "root"),
            port=int(os.getenv("PG_PORT", 5432))
        )
    except Exception as e:
        print("Erro ao conectar ao banco de dados:", e)
        raise

def create_tables():
    conn = connect_db()
    try:
        with conn.cursor() as cur:
            cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
            cur.execute("""
                CREATE TABLE IF NOT EXISTS propositions (
                    id SERIAL PRIMARY KEY,
                    proposition TEXT NOT NULL,
                    embedding vector(1536)
                );
            """)
            conn.commit()
    except Exception as e:
        print("Erro ao criar tabelas:", e)
    finally:
        conn.close()

def create_chat_history_table():
    conn = connect_db()
    try:
        with conn.cursor() as cur:
            cur.execute("""
                CREATE TABLE IF NOT EXISTS chat_history (
                    id SERIAL PRIMARY KEY,
                    user_message TEXT NOT NULL,
                    bot_response TEXT NOT NULL,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                );
            """)
            conn.commit()
    except Exception as e:
        print("Erro ao criar tabela de histórico:", e)
    finally:
        conn.close()

create_tables()
create_chat_history_table()

Configuração do modelo

In [ ]:
try:
    chat = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=openai_api_key)
    embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=openai_api_key)
except Exception as e:
    print("Erro ao inicializar os modelos da OpenAI:", e)
    raise

Funções utilitárias para vetorização, decomposição e manipulação de dados

In [ ]:
def search_similar_propositions(conn, question_embedding, top_k=3, distance_threshold=0.5):
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as cur:
        embedding_str = "[" + ",".join(map(str, question_embedding)) + "]"
        cur.execute("""
            SELECT proposition, embedding <=> %s::vector AS distance
            FROM propositions
            ORDER BY distance
            LIMIT %s;
        """, (embedding_str, top_k))
        results = cur.fetchall()
    return [row for row in results if row['distance'] <= distance_threshold]

def save_chat_history(conn, user_msg, bot_resp):
    with conn.cursor() as cur:
        cur.execute("""
            INSERT INTO chat_history (user_message, bot_response)
            VALUES (%s, %s);
        """, (user_msg, bot_resp))
        conn.commit()

def load_chat_history(conn, limit=50):
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as cur:
        cur.execute("""
            SELECT user_message, bot_response, created_at
            FROM chat_history
            ORDER BY created_at ASC
            LIMIT %s;
        """, (limit,))
        return cur.fetchall()

def load_and_clean_text(file_path):
    loader = UnstructuredLoader(file_path)
    docs = loader.load()
    full_text = " ".join([doc.page_content for doc in docs])
    cleaned_text = " ".join(line.strip() for line in full_text.splitlines() if line.strip())
    return cleaned_text

def split_into_sentences(text):
    return nltk.tokenize.sent_tokenize(text)

def decompose_sentence(sentence, chat):
    system_prompt = """
Decompose the "Content" into clear and simple propositions, ensuring they are interpretable out of context.

1. Split compound sentence into simple sentences...
...
    """.strip()

    try:
        system_message = SystemMessage(content=system_prompt.format(content=sentence))
        human_message = HumanMessage(content="")
        response = chat([system_message, human_message])
        text = response.content.strip()
        if text.startswith("```json"): text = text[len("```json"):].strip()
        if text.endswith("```"): text = text[:-3].strip()
        propositions = json.loads(text)
    except Exception:
        return [sentence]

    return [p.strip() for p in propositions if p.strip()]

def insert_embedding(conn, proposition, embedding):
    with conn.cursor() as cur:
        embedding_str = "[" + ",".join(map(str, embedding)) + "]"
        cur.execute("""
            INSERT INTO propositions (proposition, embedding)
            VALUES (%s, %s::vector);
        """, (proposition, embedding_str))
        conn.commit()

Modelos e rotas FastAPI

In [ ]:
class UserMessage(BaseModel):
    message: str

@app.get("/", response_class=HTMLResponse)
async def index():
    return "<h1>Chatbot com Upload</h1>"

@app.get("/history")
async def get_history():
    try:
        conn = connect_db()
        history = load_chat_history(conn)
        return JSONResponse(content=[{
            "user_message": h["user_message"],
            "bot_response": h["bot_response"],
            "created_at": h["created_at"].isoformat()
        } for h in history])
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Erro ao carregar histórico: {e}")
    finally:
        if conn: conn.close()

@app.post("/chat")
async def chat_endpoint(user_msg: UserMessage):
    try:
        user_input = user_msg.message
        conn = connect_db()
        question_emb = embeddings_model.embed_query(user_input)
        results = search_similar_propositions(conn, question_emb)

        if not results:
            bot_response = "Desculpe, não encontrei respostas relevantes."
        else:
            context_texts = "\n".join([f"- {row['proposition']}" for row in results])
            system_prompt = f"""
Você é um assistente que responde perguntas baseando-se unicamente nas informações listadas abaixo...

{context_texts}

Pergunta: {user_input}

Resposta:"""
            response = chat([SystemMessage(content=system_prompt), HumanMessage(content="")])
            bot_response = response.content

        save_chat_history(conn, user_input, bot_response)
        return {"response": bot_response}

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Erro durante o chat: {e}")
    finally:
        if conn: conn.close()

@app.post("/uploadfile")
async def upload_file(file: UploadFile = File(...)):
    if not file.filename:
        raise HTTPException(status_code=400, detail="Arquivo não enviado")

    temp_path = f"temp_{file.filename}"
    try:
        with open(temp_path, "wb") as buffer:
            buffer.write(await file.read())

        text = load_and_clean_text(temp_path)
        sentences = split_into_sentences(text)
        conn = connect_db()
        count = 0
        for sentence in sentences:
            decomposed_props = decompose_sentence(sentence, chat)
            for prop in decomposed_props:
                if prop:
                    emb = embeddings_model.embed_query(prop)
                    insert_embedding(conn, prop, emb)
                    count += 1
        return {"message": f"{count} proposições vetorizadas com sucesso."}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Erro ao processar o arquivo: {e}")
    finally:
        if os.path.exists(temp_path): os.remove(temp_path)
        if conn: conn.close()

Executar servidor

In [ ]:
if __name__ == "__main__":
    uvicorn.run("chatbot:app", host="127.0.0.1", port=8000, reload=True)